# 函数

In [26]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

from sklearn.ensemble import RandomForestRegressor
from SALib.sample import sobol as sobol_sample
from SALib.analyze import sobol
from sklearn.preprocessing import MinMaxScaler

def run_analysis(target_variable, feature_variables_s1_st, feature_pairs_s2, 
                 input_csv_path, output_dir_path, 
                 model_params, analysis_params, plot_config,
                 model_filter,
                 ax,
                 heatmap_colormap="Reds"):
    """
    使用三阶段训练的RandomForest模型进行Sobol敏感性分析，并将S2热力图绘制在指定的子图(ax)上。
    所有超参数通过字典传入。
    """
    # ==========================================================================
    # 1. 路径与字体配置
    # ==========================================================================
    input_csv = input_csv_path
    output_dir = output_dir_path
    
    os.makedirs(output_dir, exist_ok=True)
    
    matplotlib.rcParams['font.family'] = 'serif'
    matplotlib.rcParams['font.serif'] = ['Times New Roman']
    matplotlib.rcParams['font.sans-serif'] = ['SimSun']
    matplotlib.rcParams['axes.unicode_minus'] = False
    matplotlib.rcParams['font.weight'] = 'bold'
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    matplotlib.rcParams['axes.titleweight'] = 'bold'

    # ==========================================================================
    # 2. 数据准备
    # ==========================================================================
    print(f"--- 步骤 1/5: 正在为目标 [{target_variable.upper()}] 准备数据... ---")
    s1_vars = set(feature_variables_s1_st)
    s2_vars = set(var for pair in feature_pairs_s2 for var in pair)
    all_feature_vars = sorted(list(s1_vars.union(s2_vars)))
    print(f"    分析将针对以下 {len(all_feature_vars)} 个特征变量进行：{all_feature_vars}")
    print(f"    正在从 {os.path.basename(input_csv)} 加载数据...")
    df_raw = pd.read_csv(input_csv, encoding='utf-8-sig')

    df_all = df_raw[[target_variable] + all_feature_vars].dropna()
    X_all = df_all[all_feature_vars]
    y_all = df_all[target_variable].values
    print(f"    全局数据集准备完成，样本量为: {len(df_all)}")

    fine_tune_regions = analysis_params['fine_tune_regions']
    df_china = df_raw[df_raw['Region'].isin(fine_tune_regions)][[target_variable] + all_feature_vars].dropna()
    X_china = df_china[all_feature_vars]
    y_china = df_china[target_variable].values
    print(f"    中国区域数据集准备完成，样本量为: {len(df_china)}")

    if not model_filter or model_filter not in df_raw['Model'].unique():
        print(f"[错误] 未提供有效的目标模型名称 '{model_filter}'，任务终止。")
        return
    df_model = df_raw[df_raw['Model'] == model_filter][[target_variable] + all_feature_vars].dropna()
    X_model = df_model[all_feature_vars]
    y_model = df_model[target_variable].values
    print(f"    目标模型 '{model_filter}' 数据集准备完成，样本量为: {len(df_model)}")
    
    if len(df_all) < 20 or len(df_china) < 20 or len(df_model) < 20:
        print(f"[警告] 存在样本量过少的数据集，跳过 [{target_variable.upper()}] 任务。")
        return

    # ==========================================================================
    # 3. Sobol 敏感性分析
    # ==========================================================================
    print(f"\n--- 步骤 2/5: 正在准备 Sobol 分析... ---")
    scaler = MinMaxScaler()
    scaler.fit(X_all) 
    problem = {
        'num_vars': len(all_feature_vars),
        'names': all_feature_vars,
        'bounds': [[0, 1]] * len(all_feature_vars)
    }
    
    sobol_n_samples = analysis_params['sobol_n_samples']
    param_values = sobol_sample.sample(problem, sobol_n_samples, calc_second_order=True)
    print(f"    Sobol样本生成完毕，总样本量 (N * (2D + 2)) 为: {param_values.shape[0]} 条")

    # ==========================================================================
    # 4. 三阶段模型训练与预测
    # ==========================================================================
    print(f"\n--- 步骤 3/5: 正在为 [{target_variable.upper()}] 进行三阶段模型训练... ---")
    X_all_scaled = scaler.transform(X_all)
    X_china_scaled = scaler.transform(X_china)
    X_model_scaled = scaler.transform(X_model)
    
    rf_base_config = model_params['rf_base_config'].copy()
    rf_base_config['warm_start'] = True
    model = RandomForestRegressor(**rf_base_config)
    
    print(f"    第一阶段：使用全局样本训练 ({model_params['stage1_trees']}棵树)...")
    model.n_estimators = model_params['stage1_trees']
    model.fit(X_all_scaled, y_all)

    print(f"    第二阶段：使用中国样本微调 (额外{model_params['stage2_trees']}棵树)...")
    model.n_estimators += model_params['stage2_trees']
    model.fit(X_china_scaled, y_china)
    
    print(f"    第三阶段：使用 '{model_filter}' 样本微调 (额外{model_params['stage3_trees']}棵树)...")
    model.n_estimators += model_params['stage3_trees']
    model.fit(X_model_scaled, y_model)
    print(f"    最终模型训练完成 (总计 {model.n_estimators} 棵树)。")

    print("    正在使用最终模型对Sobol样本进行预测...")
    Y_pred = model.predict(param_values)
    print("    正在计算Sobol指数...")
    sobol_result = sobol.analyze(problem, Y_pred, calc_second_order=True, print_to_console=False)
    
    s1_st_results = pd.DataFrame({
        "变量": all_feature_vars,
        "一阶S1": np.round(sobol_result['S1'], 4),
        "总效ST": np.round(sobol_result['ST'], 4),
    }).sort_values(by="总效ST", ascending=False)
    
    s1_st_filename = f"{target_variable}_model_{model_filter.replace(' ', '_').replace('.', '')}_S1_ST.csv"
    s1_st_path = os.path.join(output_dir, s1_st_filename)
    s1_st_results.to_csv(s1_st_path, index=False, encoding='utf-8-sig')
    print(f"[✔] S1 和 ST 指数已保存至: {s1_st_path}")

    # ==========================================================================
    # 5. S2 交互效应分析与热力图可视化
    # ==========================================================================
    print(f"\n--- 步骤 4/5: 正在为 [{target_variable.upper()}] 处理 S2 指数并生成热力图... ---")
    s2_matrix = sobol_result['S2']
    var_names = problem['names']
    s2_df = pd.DataFrame(s2_matrix, index=var_names, columns=var_names)
    s2_values = []
    for pair in feature_pairs_s2:
        v1, v2 = sorted(pair)
        s2_val = s2_df.loc[v1, v2]
        s2_values.append({"特征对": f"{v1} & {v2}", "S2指数": s2_val})
    if not s2_values:
        print("[!] 未指定有效的S2特征对，跳过S2可视化。")
    else:
        s2_paired_results = pd.DataFrame(s2_values).sort_values(by="S2指数", ascending=False)
        s2_csv_filename = f"{target_variable}_model_{model_filter.replace(' ', '_').replace('.', '')}_S2_pairs.csv"
        s2_csv_path = os.path.join(output_dir, s2_csv_filename)
        s2_paired_results.to_csv(s2_csv_path, index=False, encoding='utf-8-sig')
        print(f"[✔] 指定特征对的 S2 指数已保存至: {s2_csv_path}")
        
        unique_s2_vars = sorted(list(set(var for pair in feature_pairs_s2 for var in pair)))
        s2_heatmap_df = pd.DataFrame(np.nan, index=unique_s2_vars, columns=unique_s2_vars)
        for var1, var2 in feature_pairs_s2:
            v1_s, v2_s = sorted((var1, var2))
            s2_val = s2_df.loc[v1_s, v2_s]
            s2_heatmap_df.loc[var1, var2] = s2_val
            s2_heatmap_df.loc[var2, var1] = s2_val
        
        sns.heatmap(
            s2_heatmap_df, cmap=heatmap_colormap, annot=True, fmt=".4f",
            linewidths=.5, annot_kws={'size': plot_config['annot_fontsize'], 
                                      'fontweight': 'bold'},
            cbar_kws={'label': ''}, ax=ax
        )
        
        title_text = f"{model_filter} - {target_variable.upper()}"
        ax.set_title(title_text, fontsize=plot_config['title_fontsize'])
        
        ax.tick_params(axis='x', labelsize=plot_config['tick_label_fontsize'], labelrotation=45)
        ax.tick_params(axis='y', labelsize=plot_config['tick_label_fontsize'], labelrotation=0)
        
        for label in ax.get_xticklabels(): label.set_fontweight('bold')
        for label in ax.get_yticklabels(): label.set_fontweight('bold')

        cbar = ax.collections[0].colorbar
        
        # --- 修改点: 移除了颜色条标题 ---
        # cbar.set_label('S2 Index', fontsize=plot_config['cbar_label_fontsize'])
        
        cbar.ax.tick_params(labelsize=plot_config['cbar_tick_fontsize'])
        for label in cbar.ax.get_yticklabels():
            label.set_fontweight('bold')
    
    print(f"    绘图完成: [{target_variable.upper()}]")




# 运行区域

In [29]:
# ==============================================================================
# 主程序执行区 (Main Execution Area)
# ==============================================================================
if __name__ == "__main__":
    
    input_file = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\变量归因_模型归因数据库.csv"
    output_folder = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化"

    analysis_hyperparams = {
        'sobol_n_samples': 32768,
        'fine_tune_regions': ["CHN", "R10CHINA+"]
    }
    model_hyperparams = {
        'stage1_trees': 40, 'stage2_trees': 20, 'stage3_trees': 20,
        'rf_base_config': {
            'max_depth': 8, 'min_samples_leaf': 4, 'max_features': "sqrt",
            'random_state': 42, 'n_jobs': -1
        }
    }
    plotting_parameters = {
        'figure_size': (20, 9), 'title_fontsize': 44, 'annot_fontsize': 45,
        'tick_label_fontsize': 45, 'cbar_label_fontsize': 35, 'cbar_tick_fontsize': 30
    }

    analyses_to_run = [
        {"target": "fei", "model_filter": "GCAM 5.3", "features_for_s1_st": ["seebwoc", "seec", "seegwoc"], "features_for_s2": [("seebwoc", "eced"), ("seec", "eced"), ("seegwoc", "fel")], "color_theme": "Greens", "subplot_label": "a"},
        {"target": "fet", "model_filter": "GCAM 5.3", "features_for_s1_st": ["feg", "fel", "seeb"], "features_for_s2": [("feg", "fel"), ("fel", "sees"), ("seeb", "feg")], "color_theme": "Reds", "subplot_label": "b"},
        {"target": "pe", "model_filter": "COFFEE 1.1", "features_for_s1_st": ["emcoen", "fee", "see"], "features_for_s2": [("emcoen", "fee"), ("fee", "see"), ("see", "seeh")], "color_theme": "Purples", "subplot_label": "c"}
    ]
    
    num_analyses = len(analyses_to_run)
    fig, axes = plt.subplots(nrows=num_analyses, ncols=1, figsize=(24, 12 * num_analyses))
    if num_analyses == 1: axes = [axes]

    for i, config in enumerate(analyses_to_run):
        print(f"\n{'='*30} 开始分析任务 {i+1}/{num_analyses}: 目标 {config['target'].upper()} | 模型 {config['model_filter']} {'='*30}")
        
        run_analysis(
            target_variable=config['target'],
            feature_variables_s1_st=config['features_for_s1_st'],
            feature_pairs_s2=config['features_for_s2'],
            input_csv_path=input_file,
            output_dir_path=output_folder,
            model_filter=config['model_filter'],
            ax=axes[i],
            heatmap_colormap=config['color_theme'],
            analysis_params=analysis_hyperparams,
            model_params=model_hyperparams,
            plot_config=plotting_parameters
        )
        
        axes[i].text(-0.1, 1.1, config['subplot_label'], 
                     transform=axes[i].transAxes,
                     fontsize=40, fontweight='bold', va='top', ha='left')

    print("\n[✔] 所有分析和绘图完成，正在保存最终组合图...")
    fig.tight_layout(pad=5.0)
    
    final_image_path = os.path.join(output_folder, "Combined_RF_Sobol_Analysis_All_Targets.png")
    plt.savefig(final_image_path, dpi=300)
    plt.close()

    print(f"[✔] 组合图已成功保存至: {final_image_path}")


============================== 开始分析任务 1/3: 目标 FEI | 模型 GCAM 5.3 ==============================
--- 步骤 1/5: 正在为目标 [FEI] 准备数据... ---
    分析将针对以下 5 个特征变量进行：['eced', 'fel', 'seebwoc', 'seec', 'seegwoc']
    正在从 变量归因_模型归因数据库.csv 加载数据...
    全局数据集准备完成，样本量为: 25479
    中国区域数据集准备完成，样本量为: 2547
    目标模型 'GCAM 5.3' 数据集准备完成，样本量为: 4590

--- 步骤 2/5: 正在准备 Sobol 分析... ---
    Sobol样本生成完毕，总样本量 (N * (2D + 2)) 为: 393216 条

--- 步骤 3/5: 正在为 [FEI] 进行三阶段模型训练... ---
    第一阶段：使用全局样本训练 (40棵树)...
    第二阶段：使用中国样本微调 (额外20棵树)...
    第三阶段：使用 'GCAM 5.3' 样本微调 (额外20棵树)...
    最终模型训练完成 (总计 80 棵树)。
    正在使用最终模型对Sobol样本进行预测...
    正在计算Sobol指数...


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


[✔] S1 和 ST 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\fei_model_GCAM_53_S1_ST.csv

--- 步骤 4/5: 正在为 [FEI] 处理 S2 指数并生成热力图... ---
[✔] 指定特征对的 S2 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\fei_model_GCAM_53_S2_pairs.csv
    绘图完成: [FEI]

============================== 开始分析任务 2/3: 目标 FET | 模型 GCAM 5.3 ==============================
--- 步骤 1/5: 正在为目标 [FET] 准备数据... ---
    分析将针对以下 4 个特征变量进行：['feg', 'fel', 'seeb', 'sees']
    正在从 变量归因_模型归因数据库.csv 加载数据...
    全局数据集准备完成，样本量为: 25479
    中国区域数据集准备完成，样本量为: 2547
    目标模型 'GCAM 5.3' 数据集准备完成，样本量为: 4590

--- 步骤 2/5: 正在准备 Sobol 分析... ---
    Sobol样本生成完毕，总样本量 (N * (2D + 2)) 为: 327680 条

--- 步骤 3/5: 正在为 [FET] 进行三阶段模型训练... ---
    第一阶段：使用全局样本训练 (40棵树)...
    第二阶段：使用中国样本微调 (额外20棵树)...
    第三阶段：使用 'GCAM 5.3' 样本微调 (额外20棵树)...
    最终模型训练完成 (总计 80 棵树)。
    正在使用最终模型对Sobol样本进行预测...
    正在计算Sobol指数...


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


[✔] S1 和 ST 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\fet_model_GCAM_53_S1_ST.csv

--- 步骤 4/5: 正在为 [FET] 处理 S2 指数并生成热力图... ---
[✔] 指定特征对的 S2 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\fet_model_GCAM_53_S2_pairs.csv
    绘图完成: [FET]

============================== 开始分析任务 3/3: 目标 PE | 模型 COFFEE 1.1 ==============================
--- 步骤 1/5: 正在为目标 [PE] 准备数据... ---
    分析将针对以下 4 个特征变量进行：['emcoen', 'fee', 'see', 'seeh']
    正在从 变量归因_模型归因数据库.csv 加载数据...
    全局数据集准备完成，样本量为: 25479
    中国区域数据集准备完成，样本量为: 2547
    目标模型 'COFFEE 1.1' 数据集准备完成，样本量为: 6930

--- 步骤 2/5: 正在准备 Sobol 分析... ---
    Sobol样本生成完毕，总样本量 (N * (2D + 2)) 为: 327680 条

--- 步骤 3/5: 正在为 [PE] 进行三阶段模型训练... ---
    第一阶段：使用全局样本训练 (40棵树)...
    第二阶段：使用中国样本微调 (额外20棵树)...
    第三阶段：使用 'COFFEE 1.1' 样本微调 (额外20棵树)...
    最终模型训练完成 (总计 80 棵树)。
    正在使用最终模型对Sobol样本进行预测...
    正在计算Sobol指数...


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


[✔] S1 和 ST 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\pe_model_COFFEE_11_S1_ST.csv

--- 步骤 4/5: 正在为 [PE] 处理 S2 指数并生成热力图... ---
[✔] 指定特征对的 S2 指数已保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\pe_model_COFFEE_11_S2_pairs.csv
    绘图完成: [PE]

[✔] 所有分析和绘图完成，正在保存最终组合图...
[✔] 组合图已成功保存至: C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\RF验证优化\Combined_RF_Sobol_Analysis_All_Targets.png
